DeepPavlov Classification Demonstration

In [1]:
import tensorflow
#from tensorflow import keras
#import tensorflow.keras

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd 

print("Tensorflow version {}".format(tf.__version__))
print("Numpy version {}".format(np.__version__))

Tensorflow version 1.15.0
Numpy version 1.18.0


In [3]:
pwd #'C:\Users\fghanem.FADIGHANEM\Jupyter\intent_r'

'C:\\Users\\fghanem.FADIGHANEM\\Jupyter'

In [4]:
df_train = pd.read_csv('intent_r/train_r.csv')
df_train.head()

,text,intent
0,listen to westbam alumb allergic on google music,PlayMusic
1,add step to me to the 50 clásicos playlist,AddToPlaylist
2,i give this current textbook a rating value of...,RateBook
3,play the song little robin redbreast,PlayMusic
4,please add iris dement to my playlist this is ...,AddToPlaylist


In [5]:
import os
os.listdir('intent_r')

['classes.dict', 'test_r.csv', 'tokens.dict', 'train_r.csv', 'valid_r.csv']

In [6]:
# use BasicClassificationDatasetReader to read csv data
from deeppavlov.dataset_readers.basic_classification_reader import BasicClassificationDatasetReader
# read data from particular columns of `.csv` file
dr = BasicClassificationDatasetReader().read(
    data_path= './intent_r/',
    train='train_r.csv',
    test = 'test_r.csv',
    valid = 'valid_r.csv',
    x = 'text',
    y = 'intent'  
)

In [7]:
# returns type: dict
type(dr)

dict

In [8]:
# check train/valid/test sizes
[(k, len(dr[k])) for k in dr.keys()]

[('train', 13084), ('valid', 700), ('test', 700)]

In [9]:
# use BasicClassificationDatasetIterator
from deeppavlov.dataset_iterators.basic_classification_iterator import BasicClassificationDatasetIterator

In [10]:
train_iterator = BasicClassificationDatasetIterator(data=dr)
# can enter any other data type including `all`
x_train, y_train = train_iterator.get_instances(data_type='train')
x_test, y_test = train_iterator.get_instances(data_type='test')
x_valid, y_valid = train_iterator.get_instances(data_type='valid')


In [11]:
for x, y in list(zip(x_train, y_train))[:5]:
    print('x:', x)
    print('y:', y)
    print('=================')

x: listen to westbam alumb allergic on google music
y: PlayMusic
x: add step to me to the 50 clásicos playlist
y: AddToPlaylist
x: i give this current textbook a rating value of 1 and a best rating of 6
y: RateBook
x: play the song little robin redbreast
y: PlayMusic
x: please add iris dement to my playlist this is selena
y: AddToPlaylist


In [12]:
# use str_lower tp lowercase texts.
from deeppavlov.models.preprocessors.str_lower import str_lower

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\fghanem.FADIGHANEM\AppData\Roaming\nltk_data.
[nltk_data]     ..
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\fghanem.FADIGHANEM\AppData\Roaming\nltk_data.
[nltk_data]     ..
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     C:\Users\fghanem.FADIGHANEM\AppData\Roaming\nltk_data.
[nltk_data]     ..
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     C:\Users\fghanem.FADIGHANEM\AppData\Roaming\nltk_data.
[nltk_data]     ..
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!


In [13]:
str_lower(['I wonder if DeepPavlov can make this sentence Lowercase.'])

['i wonder if deeppavlov can make this sentence lowercase.']

In [14]:
#NLTKTokenizer splits string to tokens.
from deeppavlov.models.tokenizers.nltk_moses_tokenizer import NLTKMosesTokenizer

In [15]:
tokenizer = NLTKMosesTokenizer()
tokenizer(['I wonder if DeepPavlov can make this sentence Lowercase.'])

[['I',
  'wonder',
  'if',
  'DeepPavlov',
  'can',
  'make',
  'this',
  'sentence',
  'Lowercase',
  '.']]

In [16]:
#tokenize train text data
train_x_lower_tokenized = str_lower(tokenizer(train_iterator.get_instances(data_type='train')[0]))

In [17]:
from deeppavlov.core.data.simple_vocab import SimpleVocabulary

In [18]:
# initialize simple vocabulary to collect all appeared in the dataset classes
classes_vocab = SimpleVocabulary(
    save_path='./intent_r/classes.dict',
    load_path='./intent_r/classes.dict')

2021-05-27 16:33:23.626 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from C:\Users\fghanem.FADIGHANEM\Jupyter\intent_r\classes.dict]


In [19]:
classes_vocab.fit((train_iterator.get_instances(data_type='train')[1]))
classes_vocab.save()

2021-05-27 16:33:26.816 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 101: [saving vocabulary to C:\Users\fghanem.FADIGHANEM\Jupyter\intent_r\classes.dict]


In [20]:
#class labels
list(classes_vocab.items())

[('PlayMusic', 0),
 ('GetWeather', 1),
 ('BookRestaurant', 2),
 ('RateBook', 3),
 ('SearchScreeningEvent', 4),
 ('SearchCreativeWork', 5),
 ('AddToPlaylist', 6)]

In [21]:
# also one can collect vocabulary of textual tokens appeared 2 and more times in the dataset
token_vocab = SimpleVocabulary(
    save_path='./intent_r/tokens.dict',
    load_path='./intent_r/tokens.dict',
    min_freq=2,
    special_tokens=('<PAD>', '<UNK>',),
    unk_token='<UNK>')

2021-05-27 16:33:28.474 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from C:\Users\fghanem.FADIGHANEM\Jupyter\intent_r\tokens.dict]


In [22]:
token_vocab.fit(train_x_lower_tokenized)
token_vocab.save()

2021-05-27 16:33:29.984 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 101: [saving vocabulary to C:\Users\fghanem.FADIGHANEM\Jupyter\intent_r\tokens.dict]


In [23]:
# number of tokens in dictionary
len(token_vocab)

4440

In [24]:
# 10 most common words and number of times their appeared
token_vocab.freqs.most_common()[:10]

[('the', 7054),
 ('a', 4205),
 ('in', 3275),
 ('to', 3209),
 ('for', 2896),
 ('of', 2449),
 ('i', 2132),
 ('at', 2006),
 ('play', 1826),
 ('is', 1551)]

In [25]:
token_ids = token_vocab(str_lower(tokenizer(['I wonder if DeepPavlov can make this sentence Lowercase.'])))
token_ids

[[8, 1, 236, 1, 39, 183, 20, 1, 1, 1]]

In [26]:
tokenizer(token_vocab(token_ids))

['i <UNK> if <UNK> can make this <UNK> <UNK> <UNK>']

###### Featurization (embedding)

In [27]:
# use BoWEmbedder for bag of words embedding
## bow converts text into tuples equal in length to size of vocab
### ith pos is 1 if i is in the text 0 else
#### does not account for order
import numpy as np
from deeppavlov.models.embedders.bow_embedder import BoWEmbedder

In [28]:
# initialize bag-of-words embedder giving total number of tokens
bow = BoWEmbedder(depth=token_vocab.len)
# it assumes indexed tokenized samples
bow(token_vocab(str_lower(tokenizer(['Is it freezing in Offerman, California?']))))

[array([0, 1, 0, ..., 0, 0, 0])]

In [29]:
# num tokens in in the vocabulary
sum(bow(token_vocab(str_lower(tokenizer(['Is it freezing in Offerman, California?']))))[0])

6

In [30]:
# SklearnComponent is a universal wrapper for any vecotirzer/estimator 
from deeppavlov.models.sklearn import SklearnComponent

In [31]:
# initialize TF-IDF vectorizer sklearn component with `transform` as infer method
tfidf = SklearnComponent(
    model_class="sklearn.feature_extraction.text:TfidfVectorizer",
    infer_method="transform",
    save_path='./tfidf_v0.pkl',
    load_path='./tfidf_v0.pkl',
    mode='train')

2021-05-27 16:33:42.852 INFO in 'deeppavlov.models.sklearn.sklearn_component'['sklearn_component'] at line 203: Loading model sklearn.feature_extraction.text:TfidfVectorizer from C:\Users\fghanem.FADIGHANEM\Jupyter\tfidf_v0.pkl
C:\Users\fghanem.FADIGHANEM\AppData\Roaming\Python\Python36\site-packages\sklearn\base.py:315: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.21.2 when using version 0.18.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\fghanem.FADIGHANEM\AppData\Roaming\Python\Python36\site-packages\sklearn\base.py:315: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.21.2 when using version 0.18.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
2021-05-27 16:33:42.870 INFO in 'deeppavlov.models.sklearn.sklearn_component'['sklearn_component'] at line 210: Model sklearn.feature_extraction.textTfidfVectorizer loaded  with parameters
20

In [32]:
# fit on textual train instances and save it
tfidf.fit(str_lower(train_iterator.get_instances(data_type='train')[0]))
tfidf.save()

2021-05-27 16:33:48.128 INFO in 'deeppavlov.models.sklearn.sklearn_component'['sklearn_component'] at line 109: Fitting model sklearn.feature_extraction.textTfidfVectorizer
2021-05-27 16:33:48.310 INFO in 'deeppavlov.models.sklearn.sklearn_component'['sklearn_component'] at line 241: Saving model to C:\Users\fghanem.FADIGHANEM\Jupyter\tfidf_v0.pkl


In [33]:
tfidf(str_lower(['Is it freezing in Offerman, California?']))

C:\Users\fghanem.FADIGHANEM\AppData\Roaming\Python\Python36\site-packages\sklearn\feature_extraction\text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


<1x11027 sparse matrix of type '<class 'numpy.float64'>'
	with 6 stored elements in Compressed Sparse Row format>

In [34]:
# number of tokens in the TF-IDF vocabulary
len(tfidf.model.vocabulary_)

11027

In [35]:
#GloVeEmbedder: unsupervised learning algo for obtaining vector rep for words

In [36]:
import sklearn 
import scipy

# issues with scipy used below fix
## pip install scipy==1.2.0
## pip install scikit-learn==0.18.2

print("sklearn version {}".format(sklearn.__version__))
print("scipy version {}".format(scipy.__version__))

sklearn version 0.18.2
scipy version 1.2.0


In [37]:
from deeppavlov.models.embedders.glove_embedder import GloVeEmbedder

C:\Users\fghanem.FADIGHANEM\AppData\Roaming\Python\Python36\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [39]:
from os import path

True

In [40]:
#GloVe embedding file
from deeppavlov.core.data.utils import simple_download
if path.exists("./glove.6B.100d.txt"):
    print("file already exists.")
else: 
    simple_download(url="http://files.deeppavlov.ai/embeddings/glove.6B.100d.txt", 
                destination="./glove.6B.100d.txt")

file already exists.


In [41]:
embedder = GloVeEmbedder(load_path='./glove.6B.100d.txt',
                         dim=100, pad_zero=True)

2021-05-27 16:36:45.150 INFO in 'deeppavlov.models.embedders.glove_embedder'['glove_embedder'] at line 52: [loading GloVe embeddings from `C:\Users\fghanem.FADIGHANEM\Jupyter\glove.6B.100d.txt`]


In [42]:
# output shape is (batch_size x max_num_tokens_in_the_batch x embedding_dim)
embedded_batch = embedder(str_lower(tokenizer(['Is it freezing in Offerman, California?']))) 
len(embedded_batch), len(embedded_batch[0]), embedded_batch[0][0].shape

(1, 8, (100,))

In [43]:
# True False IDF Embedder
from deeppavlov.models.embedders.tfidf_weighted_embedder import TfidfWeightedEmbedder

In [44]:
weighted_embedder = TfidfWeightedEmbedder(
    embedder=embedder,  # our GloVe embedder instance
    tokenizer=tokenizer,  # our tokenizer instance
    mean=True,  # to return one vector per sample
    vectorizer=tfidf  # our TF-IDF vectorizer
)

In [45]:
# output shape is (batch_size x  embedding_dim)
embedded_batch = weighted_embedder(str_lower(tokenizer(['Is it freezing in Offerman, California?']))) 
len(embedded_batch), embedded_batch[0].shape

C:\Users\fghanem.FADIGHANEM\AppData\Roaming\Python\Python36\site-packages\sklearn\feature_extraction\text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


(1, (100,))

###### Building the Model

In [46]:
from deeppavlov.metrics.accuracy import sets_accuracy

In [47]:
# get all train and valid data from iterator
x_train, y_train = train_iterator.get_instances(data_type="train")
x_valid, y_valid = train_iterator.get_instances(data_type="valid")

In [48]:
# initialize sklearn classifier, all parameters for classifier could be passed
cls = SklearnComponent(
    model_class="sklearn.linear_model:LogisticRegression",
    infer_method="predict",
    save_path='./logreg_v0.pkl',
    load_path='./logreg_v0.pkl',
    C=1,
    mode='train')

2021-05-27 16:38:55.461 INFO in 'deeppavlov.models.sklearn.sklearn_component'['sklearn_component'] at line 203: Loading model sklearn.linear_model:LogisticRegression from C:\Users\fghanem.FADIGHANEM\Jupyter\logreg_v0.pkl
2021-05-27 16:38:55.592 INFO in 'deeppavlov.models.sklearn.sklearn_component'['sklearn_component'] at line 210: Model sklearn.linear_model.logisticLogisticRegression loaded  with parameters
2021-05-27 16:38:55.592 WARNING in 'deeppavlov.models.sklearn.sklearn_component'['sklearn_component'] at line 216: Fitting of loaded model can not be continued. Model can be fitted from scratch.If one needs to continue fitting, please, look at `warm_start` parameter


In [49]:
# fit sklearn classifier and save it
cls.fit(tfidf(x_train), y_train)
cls.save()

2021-05-27 16:38:59.235 INFO in 'deeppavlov.models.sklearn.sklearn_component'['sklearn_component'] at line 109: Fitting model sklearn.linear_model.logisticLogisticRegression
2021-05-27 16:38:59.543 INFO in 'deeppavlov.models.sklearn.sklearn_component'['sklearn_component'] at line 241: Saving model to C:\Users\fghanem.FADIGHANEM\Jupyter\logreg_v0.pkl


In [50]:
y_valid_pred = cls(tfidf(x_valid))
print(y_valid_pred[0])

AddToPlaylist


In [51]:
# fit sklearn classifier and save it
cls.fit(tfidf(x_train), y_train)
cls.save()

2021-05-27 16:39:04.759 INFO in 'deeppavlov.models.sklearn.sklearn_component'['sklearn_component'] at line 109: Fitting model sklearn.linear_model.logisticLogisticRegression
2021-05-27 16:39:05.58 INFO in 'deeppavlov.models.sklearn.sklearn_component'['sklearn_component'] at line 241: Saving model to C:\Users\fghanem.FADIGHANEM\Jupyter\logreg_v0.pkl


In [52]:
y_valid_pred = cls(tfidf(x_valid))

In [53]:
# Let's look into obtained result
print("Text sample: {}".format(x_valid[0]))
print("True label: {}".format(y_valid[0]))
print("Predicted label: {}".format(y_valid_pred[0]))

Text sample: i d like to have this track onto my classical relaxations playlist
True label: AddToPlaylist
Predicted label: AddToPlaylist


In [54]:
# let's calculate sets accuracy (because each element is a list of labels)
sets_accuracy(np.squeeze(y_valid), y_valid_pred)

0.98

In [ ]:
# works with Tensorflow version 1.15.0
# works with Numpy version 1.18.0

In [58]:
from deeppavlov.models.classifiers.keras_classification_model import KerasClassificationModel
from deeppavlov.models.preprocessors.one_hotter import OneHotter
from deeppavlov.models.classifiers.proba2labels import Proba2Labels

In [59]:
# Intialize `KerasClassificationModel` that composes CNN shallow-and-wide network 
# (name here as`cnn_model`)
cls = KerasClassificationModel(save_path="./cnn_model_v0", 
                               load_path="./cnn_model_v0", 
                               embedding_size=embedder.dim,
                               n_classes=classes_vocab.len,
                               model_name="cnn_model",
                               text_size=15, # number of tokens
                               kernel_sizes_cnn=[3, 5, 7],
                               filters_cnn=128,
                               dense_size=100,
                               optimizer="Adam",
                               learning_rate=0.1,
                               learning_rate_decay=0.01,
                               loss="categorical_crossentropy")

2021-05-27 16:48:19.952 INFO in 'deeppavlov.models.classifiers.keras_classification_model'['keras_classification_model'] at line 216: [initializing `KerasClassificationModel` from scratch as cnn_model]


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


2021-05-27 16:48:20.453 INFO in 'deeppavlov.models.classifiers.keras_classification_model'['keras_classification_model'] at line 129: Model was successfully initialized!
Model summary:
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 15, 100)]    0                                            
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 15, 128)      38528       input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 15, 128)      64128       input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_2

In [60]:
# `KerasClassificationModel` assumes one-hotted distribution of classes per sample.
# `OneHotter` converts indices to one-hot vectors representation.
#  To obtain indices we can use our `classes_vocab` intialized and fitted above
onehotter = OneHotter(depth=classes_vocab.len, single_vector=True)

In [61]:
# Train for 10 epochs
for ep in range(10):
    for x, y in train_iterator.gen_batches(batch_size=64, 
                                           data_type="train"):
        x_embed = embedder(tokenizer(str_lower(x)))
        y_onehot = onehotter(classes_vocab(y))
        cls.train_on_batch(x_embed, y_onehot)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [62]:
# Save model weights and parameters
cls.save()

2021-05-27 16:49:58.736 INFO in 'deeppavlov.models.classifiers.keras_classification_model'['keras_classification_model'] at line 346: [saving model to cnn_model_v0_opt.json]


In [64]:
cls

In [76]:
x_valid_pred = embedder(tokenizer(str_lower(x_valid)))
print(type(x_valid_pred))
y_valid_pred = cls(x_valid_pred)#notworking

<class 'numpy.ndarray'>


ValueError: Tensor Tensor("activation_4/Sigmoid:0", shape=(?, 7), dtype=float32) is not an element of this graph.

In [74]:
prob2labels = Proba2Labels(max_proba=True)


In [75]:
# Let's look into obtained result
print("Text sample: {}".format(x_valid[0]))
print("True label: {}".format(y_valid[0]))
print("Predicted probability distribution: {}".format(dict(zip(classes_vocab.keys(), 
                                                               y_valid_pred[0]))))
print("Predicted label: {}".format(classes_vocab(prob2labels(y_valid_pred))[0]))

Text sample: i d like to have this track onto my classical relaxations playlist
True label: AddToPlaylist
Predicted probability distribution: {'PlayMusic': 'A', 'GetWeather': 'd', 'BookRestaurant': 'd', 'RateBook': 'T', 'SearchScreeningEvent': 'o', 'SearchCreativeWork': 'P', 'AddToPlaylist': 'l'}
Predicted label: PlayMusic
